In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # INFO, WARNING messages are not printed
import tensorflow as tf
import time # for throughput measurements

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
print('Number of GPUs available :', len(gpus))
if gpus:
    gpu_num = 0 # Number of the GPU to be used
    try:
        #tf.config.set_visible_devices([], 'GPU')
        tf.config.set_visible_devices(gpus[gpu_num], 'GPU')
        print('Only GPU number', gpu_num, 'used.')
        tf.config.experimental.set_memory_growth(gpus[gpu_num], True)
    except RuntimeError as e:
        print(e)

# uninstall sionna first
import sys
from pathlib import Path

sys.path.append(str(Path('..')))

from sionna.fec.ldpc import *
from sionna.utils import BinarySource 
from sionna.utils.metrics import count_block_errors
from sionna.channel import Pauli
from sionna.utils.plotting import PlotBER

from tensorflow.keras.losses import BinaryCrossentropy
from sionna.utils.metrics import compute_bler
from sionna.fec.utils import int_mod_2, row_echelon


Number of GPUs available : 1
Only GPU number 0 used.


In [2]:
class BP4_Error_Model(tf.keras.Model):
# For storing error strings that the BP decoder failed to decode
    def __init__(self, code, decoder, num_iter=32, trainable=False, loss_type="boxplus-phi", wt=False):

        super().__init__()
        
        self.k = code.K      
        self.n = code.N
        self.hx = code.hx
        self.hz = code.hz
        self.lx = code.lx
        self.lz = code.lz
        self.hx_perp = code.hx_perp # contains Im(hz.T)
        self.hz_perp = code.hz_perp # contains Im(hx.T)
        self.code_name = code.name
        self.num_checks = code.hx.shape[0] + code.hz.shape[0]
        
        self.source = BinarySource()
        self.channel = Pauli(wt=wt)
        self.decoder = decoder

        self.num_iter = num_iter
        self.trainable = trainable
        self.bce = BinaryCrossentropy(from_logits=True)
        self.loss_type = loss_type
        self.wt = wt
           
#     @tf.function(jit_compile=True, reduce_retracing=True) # XLA mode
    @tf.function() # graph mode
    def call(self, batch_size, ebno_db):

        p = ebno_db

        # depolarizing noise
        px, py, pz = 2*p/3, p/3, 2*p/3
        c_dummy = tf.zeros([batch_size, self.n])
        if self.wt: # ebno_db is an integer indicating the weight of the error
            noise_x, noise_z = self.channel([c_dummy, None, p])
        else:
            noise_x, noise_z = self.channel([c_dummy, None, px, py, pz])  # [bs, self.n]
        noise_x_T, noise_z_T = tf.transpose(noise_x, (1,0)), tf.transpose(noise_z, (1,0))
        noise_x_int = tf.cast(noise_x_T, self.hz.dtype)
        noise_z_int = tf.cast(noise_z_T, self.hx.dtype)
        syndrome_x = int_mod_2(tf.matmul(self.hx, noise_z_int))
        syndrome_z = int_mod_2(tf.matmul(self.hz, noise_x_int))

        p0 = 0.05 
        llr_ch_x = tf.fill(tf.shape(noise_x), tf.math.log(3.*(1.-p0)/p0))
        llr = tf.tile(tf.expand_dims(llr_ch_x, axis=1), multiples=tf.constant([1, 3, 1], tf.int32))
        # shape of llr: [bs, 3, self.n]
        
        x_hat, z_hat = self.decoder((llr, syndrome_x, syndrome_z))
        
        x_hat = tf.transpose(tf.cast(x_hat, tf.bool), (1,0)) # [self.n, bs]
        z_hat = tf.transpose(tf.cast(z_hat, tf.bool), (1,0))

        x_diff = tf.cast(tf.math.logical_xor(noise_x_T, x_hat), self.hx_perp.dtype)
        z_diff = tf.cast(tf.math.logical_xor(noise_z_T, z_hat), self.hz_perp.dtype)

        sx = int_mod_2(tf.matmul(self.hz, x_diff))
        sz = int_mod_2(tf.matmul(self.hx, z_diff))
        s_hat = tf.concat([sx, sz], axis=0)
        s_hat = tf.transpose(s_hat, (1,0))
        err = tf.reduce_any(tf.not_equal(tf.zeros_like(s_hat), s_hat), axis=-1)
        return noise_x[err], noise_z[err]


In [24]:
GHP_n882_k24 = create_QC_GHP_codes(63, create_cyclic_permuting_matrix(7, [27,54,0]), [0,1,6]) # 18 <= d <= 24
code = GHP_n882_k24
# GHP_n1270_k28 = create_QC_GHP_codes(127, np.array([[0,-1,51,52,-1],[-1,0,-1,111,20],[0,-1,98,-1,122],[0,80,-1,119,-1],[-1,0,5,-1,106]]), [0,1,7], name="GHP_n1270_k28") # 16 <= d <= 46
# code = GHP_n1270_k28
num_iter = tf.constant(100)
# decoder = QLDPCBPDecoder(code=code, num_iter=num_iter, normalization_factor=1.0, cn_type="boxplus-phi")
decoder = QLDPCBPDecoder(code=code, num_iter=num_iter, normalization_factor=0.9, cn_type="minsum")
model = BP4_Error_Model(code, decoder, num_iter=num_iter, trainable=False, wt=False)



In [25]:
# Run this cell multiple times
batch_size = tf.constant(50000)
p = 0.08

print(f"physical error rate {p}, find BP failed-to-decode samples (flagged errors)")

sample_x_list = []
sample_z_list = []

noise_x, noise_z = model(batch_size, tf.constant(p))
sample_x_list.append(noise_x)
sample_z_list.append(noise_z)

        

physical error rate 0.08, find BP failed-to-decode samples (flagged errors)


In [26]:

decoder1 = QLDPCBPDecoder(code=code, num_iter=num_iter, normalization_factor=1.0, cn_type="boxplus-phi", trainable=False, stage_one=True)
noise_x, noise_z = sample_x_list[0], sample_z_list[0]
tf.print(tf.shape(noise_x))
noise_x_T, noise_z_T = tf.transpose(noise_x, (1,0)), tf.transpose(noise_z, (1,0))
noise_x_int = tf.cast(noise_x_T, code.hz.dtype)
noise_z_int = tf.cast(noise_z_T, code.hx.dtype)
syndrome_x = int_mod_2(tf.matmul(code.hx, noise_z_int))
syndrome_z = int_mod_2(tf.matmul(code.hz, noise_x_int))
tf.print(tf.shape(syndrome_x))
p0 = 0.05 
llr_ch_x = tf.fill(tf.shape(noise_x), tf.math.log(3.*(1.-p0)/p0))
llr = tf.tile(tf.expand_dims(llr_ch_x, axis=1), multiples=tf.constant([1, 3, 1], tf.int32))

llrx, llry, llrz, x_hat_stage_one, z_hat_stage_one, logit_hx_perp, logit_hz_perp = decoder1((llr, syndrome_x, syndrome_z))


[1013 882]
[441 1013]


In [27]:
num_hx = tf.math.softplus(-1.*llrx) # I or X, both commute with X-type check
denom_hx = tf.ragged.map_flat_values(lambda x, y: tf.math.reduce_logsumexp(-1.*tf.stack([x, y], axis=-1), axis=-1), llrz, llry)
new_llr_z = num_hx - denom_hx
tf.print(tf.shape(new_llr_z))

num_hz = tf.math.softplus(-1.*llrz) # I or Z, both commute with Z-type check
denom_hz = tf.ragged.map_flat_values(lambda x, y: tf.math.reduce_logsumexp(-1.*tf.stack([x, y], axis=-1), axis=-1), llrx, llry)
new_llr_x = num_hz - denom_hz
tf.print(tf.shape(new_llr_x))

[1013 882]
[1013 882]


In [28]:
def osd0(llr, pcm, s):
    # use llrz together with code.hx and syndrome_x, return noise_z_hat
    sort_order = tf.argsort(llr)

    permuted_pcm = tf.gather(pcm, sort_order, axis=1) 

    inv_sort = tf.math.invert_permutation(sort_order)
    
    row_ech_form, rank, transform, pivot_cols = row_echelon(permuted_pcm.numpy())
    
    new_pcm = tf.gather(permuted_pcm, pivot_cols, axis=1)
    
    syndrome = tf.expand_dims(s, 1)
    
    pcm_synd = tf.concat((new_pcm, syndrome), axis=1)
    
    row_ech_form, rank, _, _ = row_echelon(pcm_synd.numpy(), reduced=True)
    
#     tf.print(tf.shape(pivot_cols))
#     tf.print(tf.shape(row_ech_form[:,-1]))
    
    e_hat = tf.tensor_scatter_nd_update(tf.zeros_like(llr, dtype=tf.bool), tf.expand_dims(pivot_cols, axis=1), row_ech_form[:,-1])
    
    e_hat = tf.gather(e_hat, inv_sort)
    
    return e_hat

In [29]:
_, _, _, pivot_hx = row_echelon(code.hx.T)
reduced_hx = tf.gather(code.hx, pivot_hx, axis=0) # select linearly independent rows
tf.print(tf.shape(reduced_hx))

_, _, _, pivot_hz = row_echelon(code.hz.T)
reduced_hz = tf.gather(code.hz, pivot_hz, axis=0) # select linearly independent rows
tf.print(tf.shape(reduced_hz))

reduced_sx = tf.gather(syndrome_x, pivot_hx, axis=0)
reduced_sz = tf.gather(syndrome_z, pivot_hz, axis=0)

tf.print(tf.shape(reduced_sx))
tf.print(tf.shape(reduced_sz))

[429 882]
[429 882]
[429 1013]
[429 1013]


In [30]:
noise_z_hat_list, noise_x_hat_list = [], []
bs = tf.shape(new_llr_z)[0]
for i in range(bs):
    noise_z_hat_list.append(osd0(new_llr_z[i], reduced_hx, reduced_sx[:,i]))
    noise_x_hat_list.append(osd0(new_llr_x[i], reduced_hz, reduced_sz[:,i]))

In [31]:
noise_x_hat = tf.stack(noise_x_hat_list, axis=0)
tf.print(tf.shape(noise_x_hat))
noise_z_hat = tf.stack(noise_z_hat_list, axis=0)
tf.print(tf.shape(noise_z_hat))

[1013 882]
[1013 882]


In [32]:
x_hat = tf.transpose(noise_x_hat, (1,0)) # [self.n, bs]
z_hat = tf.transpose(noise_z_hat, (1,0))

print(tf.shape(noise_x_T))
print(tf.shape(x_hat))

x_diff = tf.cast(tf.math.logical_xor(noise_x_T, x_hat), code.hx_perp.dtype)
z_diff = tf.cast(tf.math.logical_xor(noise_z_T, z_hat), code.hz_perp.dtype)

sx = int_mod_2(tf.matmul(code.hz, x_diff))
sz = int_mod_2(tf.matmul(code.hx, z_diff)) 
s_hat = tf.concat([sx, sz], axis=0)
s_hat = tf.transpose(s_hat, (1,0))
err = tf.reduce_any(tf.not_equal(tf.zeros_like(s_hat), s_hat), axis=-1)
# OSD should clean up all flag errors

lsx = int_mod_2(tf.matmul(code.hx_perp, x_diff))
lsz = int_mod_2(tf.matmul(code.hz_perp, z_diff))

ls_hat = tf.concat([lsx, lsz], axis=0)      # for total logical error counts
ls_hat = tf.transpose(ls_hat, (1,0))         # bs should be the first dimension!!!
logical_err = tf.reduce_any(tf.not_equal(tf.zeros_like(ls_hat), ls_hat), axis=-1)

tf.Tensor([ 882 1013], shape=(2,), dtype=int32)
tf.Tensor([ 882 1013], shape=(2,), dtype=int32)


In [33]:
print(tf.shape(s_hat))
print(tf.reduce_sum(s_hat))
print(tf.reduce_sum(tf.cast(logical_err, dtype=tf.int32)))

tf.Tensor([1013  882], shape=(2,), dtype=int32)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int32)


BP: flooding, sum-product, no normalization.
| CN update       | $p$  | $p_L$  |
| --------------- | :-:  | :-:    |
| SP, 1.0, 64     | 0.1  | 5.1e-3
| ^^              | 0.09 | 1.74e-3
| ^^              | 0.08 | 1.08e-3
| MS, 1.0, 64     | 0.1 | 4.74e-3
| NMS, 0.625, 100 | 0.08 | 5.4e-4
| NMS, 0.8, 100   | 0.08 | 4e-5 (2/50000)
| NMS, 0.9, 100   | 0.08 | 8e-5 (4/50000)

In [4]:
# Generalized Bicycle Codes
GB_n254_k28 = create_generalized_bicycle_codes(127, [0,15,20,28,66], [0,58,59,100,121]) # 14 <= d <= 20
GB_n126_k28_d8 = create_generalized_bicycle_codes(63, [0,1,14,16,22], [0,3,13,20,42], name="GB_n126_k28_d8")

GB_n48_k6_d8 = create_generalized_bicycle_codes(24, [0,2,8,15], [0,2,12,17], name="GB_n48_k6_d8")
# create the code using the overcomplete check matrices in [2]
GB_n48_pcm = readAlist('../sionna/fec/ldpc/codes_q/GB_48_6_H_2000.alist')
GB_n48_k6_d8_oc = css_code(hx=GB_n48_pcm[:1000], hz=GB_n48_pcm[1000:], name=None, name_prefix="GB")
GB_n48_k6_d8_oc.name += "_oc"

GB_n46_k2_d9 = create_generalized_bicycle_codes(23, [0,5,8,12], [0,1,5,7], name="GB_n46_k2_d9")
# create the code using the overcomplete check matrices in [2]
GB_n46_pcm = readAlist('../sionna/fec/ldpc/codes_q/GB_46_2_H_800.alist')
GB_n46_k2_d9_oc = css_code(hx=GB_n46_pcm[:400], hz=GB_n46_pcm[400:], name=None, name_prefix="GB")
GB_n46_k2_d9_oc.name += "_oc"

GB_n180_n10 = create_generalized_bicycle_codes(90, [0,28,80,89], [0,2,21,25]) # 15 <= d <= 18
GB_n900_k50_d15 = create_generalized_bicycle_codes(450, [0,97,372,425], [0,50,265,390], name="GB_n900_k50_d15")

# Hypergraph Product Codes
# HP_n7935_n578_H = create_circulant_matrix(63, [0,3,34,41,57]) 
# HP_n7935_n578_d16 = hypergraph_product(HP_n7935_n578_H, HP_n7935_n578_H, name="HP_n7935_n578_d16") # takes 40 seconds
HP_n1922_k50_H = create_circulant_matrix(31, [0,2,5])
HP_n1922_n50_d16 = hypergraph_product(HP_n1922_k50_H, HP_n1922_k50_H, name="HP_n1922_k50_d16")

# Generalized Hypergraph Product Codes
GHP_n882_k24 = create_QC_GHP_codes(63, create_cyclic_permuting_matrix(7, [27,54,0]), [0,1,6]) # 18 <= d <= 24
GHP_n882_k48_d16 = create_QC_GHP_codes(63, create_cyclic_permuting_matrix(7, [27,0,27,18,0]), [0,1,6], name="GHP_n882_k48_d16")
GHP_n1270_k28 = create_QC_GHP_codes(127, np.array([[0,-1,51,52,-1],[-1,0,-1,111,20],[0,-1,98,-1,122],[0,80,-1,119,-1],[-1,0,5,-1,106]]), [0,1,7], name="GHP_n1270_k28") # 16 <= d <= 46


In [5]:
code = GHP_n882_k24
decoder = LDPCBPDecoder(code.hx, is_syndrome=True, num_iter=64) # binary syndrome BP decoder
# noise added by BSC channel
model = BP_BSC_Model(pcm=code.hx, decoder=decoder, logical_pcm=code.hz_perp, p0=0.2) # p0 is used to initialize BP
# If p0 is not specified (default set to None), then the actual p value employed for noise generation will be used for BP initialization.
p_range = np.arange(0.01,0.101,0.01)[::-1] * 2/3 

ber_plot = PlotBER("Performance of the [[882,24]] code on BSC channel under binary decoding")

ber_plot.simulate(model, 
                  ebno_dbs=p_range, # physical error rates to simulate
                  legend=f"{code.name}, factor={1.0}, iter={64}, p0=0.2", # legend string for plotting
                  max_mc_iter=1000, # run 1000 Monte Carlo runs per physical error rate point
                  num_target_block_errors=100, # continue with next physical error rate point after 1000 block errors
                  batch_size=10000, # batch-size per Monte Carlo run
                  soft_estimates=False, # the model returns hard-estimates
                  early_stop=True, # stop simulation if no error has been detected at current physical error rate
                  show_fig=False, # do not show the figure after all results are simulated
                  add_bler=True, # we are interested in block error rate
                  qldpc=True, # show flagged error rate, instead of showing bit error rate
                  forward_keyboard_interrupt=True); # should be True in a loop

        p |    Flagged |       BLER | flag errors | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
    0.067 | 8.6010e-01 | 8.6010e-01 |        8601 |         8601 |       10000 |         2.2 |reached target block errors
     0.06 | 6.2610e-01 | 6.2610e-01 |        6261 |         6261 |       10000 |         0.1 |reached target block errors
    0.053 | 3.0940e-01 | 3.0940e-01 |        3094 |         3094 |       10000 |         0.1 |reached target block errors
    0.047 | 8.1000e-02 | 8.1000e-02 |         810 |          810 |       10000 |         0.1 |reached target block errors
     0.04 | 8.8000e-03 | 8.8000e-03 |         176 |          176 |       20000 |         0.2 |reached target block errors
    0.033 | 5.6667e-04 | 5.6667e-04 |         102 |          102 |      180000 |         2.2 |reached target block errors
    0.027 | 2.8571e-04 | 2.